# Compare Model Hyperparameters

In [ ]:
! git clone https://github.com/MaxHReinhardt/ArchitecturalStyleClassification

In [ ]:
import sys
import os

py_file_location = "/content/ArchitecturalStyleClassification/src"
sys.path.append(os.path.abspath(py_file_location))

In [ ]:
import torch

from preprocessing import TrainSetDynamicNormalization, EvaluationSetDynamicNormalization
from model import MobileNetV1
from train_model import train_with_early_stopping
from evaluate_model import evaluate


def compare_model_hyperparameter_configurations(width_multiplier_list, resolution_list, with_cbam_variant_list, train_csv, validation_csv):
    batch_size = 64
    learning_rate = 0.003
    max_num_epochs = 100

    # Check if CUDA (GPU) is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    for width_multiplier in width_multiplier_list:
        for resolution in resolution_list:
            for with_cbam in with_cbam_variant_list:
                # tiny datasets only for testing purposes
                train_set = TrainSetDynamicNormalization(resolution=resolution,
                                                         train_csv=train_csv).get_data()
                validation_set = EvaluationSetDynamicNormalization(resolution=resolution,
                                                                   validation_csv=validation_csv).get_data()

                model = MobileNetV1(ch_in=3, n_classes=25, width_multiplier=width_multiplier, with_cbam=with_cbam)
                model.to(device)

                trained_model, train_loss_development, val_loss_development = train_with_early_stopping(model,
                                                                                                        train_set,
                                                                                                        validation_set,
                                                                                                        batch_size,
                                                                                                        learning_rate,
                                                                                                        max_num_epochs,
                                                                                                        device)

                model_name = f"{width_multiplier}-MobileNetV1-{resolution}_{'with_cbam' if with_cbam else ''}"
                model_path = os.path.join("stored_models/", model_name + ".pth")
                torch.save(trained_model.state_dict(), model_path)

                accuracy, _, avg_loss, avg_prediction_time = evaluate(trained_model,
                                                                      validation_set,
                                                                      batch_size,
                                                                      device)

                print(f"{model_name} evaluation -- Accuracy: {accuracy}, Average loss: {avg_loss}, "
                      f"Average prediction time (seconds): {avg_prediction_time}")

In [ ]:
!mkdir stored_models

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!unzip /content/gdrive/MyDrive/data.zip;

In [ ]:
drive.flush_and_unmount()

## Run Experiment

In [ ]:
width_multiplier_list = [1, 0.75, 0.5]
resolution_list = [384]
with_cbam_variant_list = [True]
train_csv = "data/dataset/train_annotation.csv"
validation_csv = "data/dataset/validation_annotation.csv"

In [ ]:
compare_model_hyperparameter_configurations(width_multiplier_list, resolution_list, with_cbam_variant_list, train_csv, validation_csv)